In [ ]:
from mongo_episode import Episodes
import pandas as pd


def get_episodes():
    episodes = Episodes()
    all_episodes = episodes.episodes
    episodes_df = pd.DataFrame([episode.to_dict() for episode in all_episodes])
    episodes_df["date"] = episodes_df["date"].dt.strftime("%Y/%m/%d")
    episodes_df["duree (min)"] = (episodes_df["duree"] / 60).round(1)
    episodes_df.drop(
        columns=["url_telechargement", "audio_rel_filename", "type", "duree"],
        inplace=True,
    )
    return episodes_df


def print_episodes_info(episodes_df):
    print("### Informations sur les épisodes")
    print(f"{episodes_df.shape[0]} épisodes")
    print(f"{episodes_df['date'].min()} - {episodes_df['date'].max()}")
    # Compter les transcriptions disponibles et manquantes
    transcriptions_ok = episodes_df["transcription"].notna().sum()
    transcriptions_missing = episodes_df["transcription"].isna().sum()

    print(f"Transcriptions OK : {transcriptions_ok}")
    print(f"Transcriptions manquantes : {transcriptions_missing}")


def afficher_episodes():
    print("### Épisodes")
    print("Liste des épisodes du Masque et la Plume.")
    episodes_df = get_episodes()
    print_episodes_info(episodes_df)
    # Ajoutez ici le code pour afficher les épisodes
    return episodes_df

In [ ]:
episodes = Episodes()

In [ ]:
afficher_episodes()

### Épisodes
Liste des épisodes du Masque et la Plume.
### Informations sur les épisodes
206 épisodes
1958/12/18 - 2025/01/26
Transcriptions OK : 206
Transcriptions manquantes : 0


,date,titre,description,transcription,duree (min)
0,2025/01/26,"Les nouveaux romans de Leïla Slimani, Pierre L...",durée : 00:47:56 - Le Masque et la Plume - par...,France Inter Le masque et la plume Le sommair...,47.9
1,2025/01/12,"Les nouvelles pages de Vanessa Springora, Haru...",durée : 00:48:41 - Le Masque et la Plume - par...,France Inter Le masque et la plume Premier ma...,48.7
2,2024/12/22,"Les nouvelles pages de Marc Dugain, Emmanuelle...",durée : 00:47:52 - Le Masque et la Plume - par...,France Inter Le masque et la plume Cinq roman...,47.9
3,2024/12/01,"Les nouvelles pages de James Ellroy, Yasmina R...",durée : 00:47:17 - Le Masque et la Plume - par...,France Inter Le masque et la plume Les nouvea...,47.3
4,2024/11/10,La foire du livre de Brive : les romans de Dan...,durée : 00:46:03 - Le Masque et la Plume - par...,France Inter Le masque et la plume Bienvenue ...,46.0
...,...,...,...,...,...
201,1961/11/16,"Le Masque et les Goncourt : ""La Pitié de Dieu""...",Le\n journaliste et écrivain Jean Cau répond a...,France Inter Ça a été votre disparition il y ...,9.0
202,1961/11/16,"Le Masque et les Goncourt : ""L'Etat sauvage"" d...","En\n 1964, les critiques du ""Masque et la plum...",France Inter en effet il y a plusieurs mois e...,5.0
203,1960/12/15,"Le Masque et les Goncourt : ""Dieu est né en ex...","En\n 1960, le prix Goncourt est attribué à Vin...",France Inter Qu'il s'agit encore une fois de ...,1.0
204,1959/11/19,"Le Masque et les Goncourt : ""Le Dernier des ju...","Un\n livre extraordinairement émouvant, ""Le De...",France Inter Commençons par le grand cours qu...,8.0


In [ ]:
from datetime import datetime
import locale

# Définir la locale en français
locale.setlocale(locale.LC_TIME, "fr_FR.UTF-8")

DATE_FORMAT = "%d %b %Y"
episodes[0].to_dict().get("date").strftime(DATE_FORMAT)

'26 janv. 2025'

In [ ]:
selected_date = "2025/01/26"
# convert to datetime
selected_date_dt = datetime.strptime(selected_date, "%Y/%m/%d")
# convert to string
selected_date_str = selected_date_dt.strftime(DATE_FORMAT)

selected_date_str

'26 janv. 2025'

In [ ]:
import plotly.express as px


def nb_mots_transcription(episodes_df):
    print("### Nombre de mots par transcription")
    # Compter le nombre de mots dans chaque transcription

    episodes_df = episodes_df.copy()

    # Calculer le nombre de mots par minute
    episodes_df["mots_par_minute"] = (
        episodes_df["transcription"].apply(lambda x: len(x.split()))
        / episodes_df["duree (min)"]
    )

    # Transformer la date en format dd/mm/yyyy
    episodes_df["date_formatted"] = episodes_df["date"].apply(
        lambda x: datetime.strptime(x, "%Y/%m/%d")
    )
    episodes_df["date_formatted"] = episodes_df["date_formatted"].apply(
        lambda x: x.strftime("%d/%m/%Y")
    )

    # Trier les données par nombre de mots par minute décroissant
    episodes_df = episodes_df.sort_values(by="mots_par_minute", ascending=False)

    # Créer le graphique interactif
    fig = px.bar(
        episodes_df,
        x="date_formatted",
        y="mots_par_minute",
        custom_data=["date_formatted", "titre", "mots_par_minute", "duree (min)"],
        labels={"mots_par_minute": "Mots par Minute"},
        title="Nombre de Mots par Minute par Épisode",
    )

    # # Mettre à jour les informations de survol
    fig.update_traces(
        hovertemplate="<b>Date:</b> %{customdata[0]}<br><b>Titre:</b> %{customdata[1]}<br><b>Mots par Minute:</b> %{customdata[2]:.1f}<br><b>Duree en Minute:</b> %{customdata[3]:.1f}"
    )

    # Afficher le graphique
    fig.show()

In [ ]:
episodes_df = get_episodes()

nb_mots_transcription(episodes_df)

### Nombre de mots par transcription
